In [8]:
import os
import sys
from functools import partial
from pathlib import Path
path = os.path.join(os.getcwd(), '..')
sys.path.append(path)

import warnings 
warnings.filterwarnings("ignore")

from fastai2.vision.all import *

from src.data.dls import build_dataloaders
from src.model.FasterRCNN import get_faster_rcnn, split_faster_rcnn_params
from src.metrics.loss import WeightedMultiLoss
from src.metrics.mAP import mAP
from src.callback.core_cbs import FasterRCNNCallback, CheckpointCallback
from src.callback.core_cbs import after_batch, begin_validate
from src.callback.learner import WheatLearner

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### 0. Parameters

In [9]:
DATA_PATH = Path('/userhome/34/h3509807/wheat-data')
RESIZE_SZ = 256
RAND_SEED = 144
TEST_MODE = False
BS = 8

#### 1. Setup Learner and Load Trained Learner

In [10]:
get_dls = partial(build_dataloaders, data_path = DATA_PATH, 
                  resize_sz = RESIZE_SZ, norm = False, 
                  rand_seed = RAND_SEED, test_mode = TEST_MODE)
dls = get_dls(bs = BS)

model = get_faster_rcnn()

multi_loss = WeightedMultiLoss()
thresholds = [i for i in map(lambda i: i/100, range(50, 80, 5))]
mAP_calc = partial(mAP, thresholds = thresholds)

faster_rcnn_cb = FasterRCNNCallback(img_size = RESIZE_SZ)

In [22]:
learn = WheatLearner(dls, model, loss_func = multi_loss, 
                     splitter = split_faster_rcnn_params,
                     metrics = mAP_calc,
                     cbs = faster_rcnn_cb)
learn.path = Path('./..')

In [26]:
learn = learn.load('final_learner')

#### 2. Do Inference
- TfmdList.show_batch has issue i.e. b is a tuple (TensorImage, dict, dict, ...)

In [27]:
learn.show_results()

RuntimeError: Could not infer dtype of dict